In [1]:
!pip install deeprobust

     |████████████████████████████████| 184kB 7.9MB/s 
     |████████████████████████████████| 24.2MB 106kB/s 
     |████████████████████████████████| 122kB 48.8MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


# Global Attack for Node classification

In [3]:
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import Metattack
data = Dataset(root='/tmp/', name='cora')
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)
idx_unlabeled = np.union1d(idx_val, idx_test)
# Setup Surrogate model
surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, dropout=0, with_relu=False, with_bias=False, device='cpu').to('cpu')
surrogate.fit(features, adj, labels, idx_train, idx_val, patience=30)
# Setup Attack Model
model = Metattack(surrogate, nnodes=adj.shape[0], feature_shape=features.shape,
        attack_structure=True, attack_features=False, device='cpu', lambda_=0).to('cpu')
# Attack
model.attack(features, adj, labels, idx_train, idx_unlabeled, n_perturbations=10, ll_constraint=False)
modified_adj = model.modified_adj # modified_adj is a torch.tensor

No module named 'torch_geometric'
Loading cora dataset...
Selecting 1 largest connected components


/usr/local/lib/python3.7/dist-packages/deeprobust/graph/defense/__init__.py:16: UserWarning: Please install pytorch geometric if you would like to use the datasets from pytorch geometric. See details in https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html
  "geometric. See details in https://pytorch-geom" +
Perturbing graph:   0%|          | 0/10 [00:00<?, ?it/s]

GCN loss on unlabled data: 0.5084404945373535
GCN acc on unlabled data: 0.8301296379079124
attack loss: 0.3048456311225891


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
Perturbing graph:  10%|█         | 1/10 [00:15<02:22, 15.87s/it]

GCN loss on unlabled data: 0.5220595598220825
GCN acc on unlabled data: 0.8421993741618239
attack loss: 0.3358333706855774


...

Perturbing graph:  80%|████████  | 8/10 [02:05<00:31, 15.67s/it]

GCN loss on unlabled data: 0.540736973285675
GCN acc on unlabled data: 0.8314707197139025
attack loss: 0.34770819544792175


Perturbing graph:  90%|█████████ | 9/10 [02:21<00:15, 15.68s/it]

GCN loss on unlabled data: 0.5553375482559204
GCN acc on unlabled data: 0.8265534197586053
attack loss: 0.3598272204399109


Perturbing graph: 100%|██████████| 10/10 [02:36<00:00, 15.70s/it]


# Targeted Attack for Node Classification

In [4]:
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.targeted_attack import Nettack
data = Dataset(root='/tmp/', name='cora')
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
# Setup Surrogate model
surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, dropout=0, with_relu=False, with_bias=False, device='cpu').to('cpu')
surrogate.fit(features, adj, labels, idx_train, idx_val, patience=30)
# Setup Attack Model
target_node = 0
model = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=True, device='cpu').to('cpu')
# Attack
model.attack(features, adj, labels, target_node, n_perturbations=5)
modified_adj = model.modified_adj # scipy sparse matrix
modified_features = model.modified_features # scipy sparse matrix

Loading cora dataset...
Selecting 1 largest connected components
##### Starting attack #####
##### Attack node with ID 0 using structure and feature perturbations #####
##### Attacking the node directly #####
##### Performing 5 perturbations #####
##### ...1/5 perturbations ... #####


/usr/local/lib/python3.7/dist-packages/numba/core/ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'edges_set' of function 'compute_new_a_hat_uv'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../usr/local/lib/python3.7/dist-packages/deeprobust/graph/targeted_attack/nettack.py", line 501:
@jit(nopython=True)
def compute_new_a_hat_uv(edge_ixs, node_nb_ixs, edges_set, twohop_ixs, values_before, degs, potential_edges, u):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


Edge perturbation: [   0 1664]
##### ...2/5 perturbations ... #####
Edge perturbation: [   0 1301]
##### ...3/5 perturbations ... #####
Edge perturbation: [   0 1084]
##### ...4/5 perturbations ... #####
Edge perturbation: [   0 1193]
##### ...5/5 perturbations ... #####
Edge perturbation: [   0 1046]


## 1. Load pre-attacked graph data

In [5]:
from deeprobust.graph.data import Dataset, PrePtbDataset
# load the prognn splits by using setting='prognn'
# because the attacked graphs are generated under prognn splits
data = Dataset(root='/tmp/', name='cora', setting='prognn')

adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
# Load meta attacked data
perturbed_data = PrePtbDataset(root='/tmp/',
                    name='cora',
                    attack_method='meta',
                    ptb_rate=0.05)
perturbed_adj = perturbed_data.adj

Loading cora dataset...
Selecting 1 largest connected components
Dowloading from https://raw.githubusercontent.com/ChandlerBang/Pro-GNN/master/splits/cora_prognn_splits.json to /tmp/cora_prognn_splits.json
Dowloading from https://raw.githubusercontent.com/ChandlerBang/Pro-GNN/master/meta/cora_meta_adj_0.05.npz to /tmp/cora_meta_adj_0.05.npz
Loading cora dataset perturbed by 0.05 meta...


/usr/local/lib/python3.7/dist-packages/deeprobust/graph/data/attacked_data.py:135: UserWarning: The pre-attacked graph is perturbed under the data splits provided by ProGNN. So if you are going to verify the attacking performance, you should use the same data splits  (setting='prognn').
  warnings.warn("The pre-attacked graph is perturbed under the data splits provided by ProGNN. So if you are going to verify the attacking performance, you should use the same data splits  (setting='prognn').")


## 2. Train victim model on clean/poisoned graph

In [6]:
from deeprobust.graph.defense import GCN
gcn = GCN(nfeat=features.shape[1],
    nhid=16,
    nclass=labels.max().item() + 1,
    dropout=0.5, device='cpu')
gcn = gcn.to('cpu')
gcn.fit(features, adj, labels, idx_train, idx_val) # train on clean graph with earlystopping
gcn.test(idx_test)

gcn.fit(features, perturbed_adj, labels, idx_train, idx_val) # train on poisoned graph
gcn.test(idx_test)

Test set results: loss= 0.5641 accuracy= 0.8290
Test set results: loss= 0.7253 accuracy= 0.7762


0.7761569416498993

## 3. Train defense models

In [7]:
from deeprobust.graph.defense import GCNJaccard
model = GCNJaccard(nfeat=features.shape[1],
          nhid=16,
          nclass=labels.max().item() + 1,
          dropout=0.5, device='cpu').to('cpu')
model.fit(features, perturbed_adj, labels, idx_train, idx_val, threshold=0.03)
model.test(idx_test)

removed 1191 edges in the original graph
=== training gcn model ===
Epoch 0, training loss: 1.907806634902954
Epoch 10, training loss: 1.1862224340438843
Epoch 20, training loss: 0.5860435366630554
Epoch 30, training loss: 0.3268839716911316
Epoch 40, training loss: 0.18484485149383545
Epoch 50, training loss: 0.14373797178268433
Epoch 60, training loss: 0.13396364450454712
Epoch 70, training loss: 0.10623839497566223
Epoch 80, training loss: 0.08720427751541138
Epoch 90, training loss: 0.06903304904699326
Epoch 100, training loss: 0.09412094205617905
Epoch 110, training loss: 0.0924135223031044
Epoch 120, training loss: 0.08030423521995544
Epoch 130, training loss: 0.07552876323461533
Epoch 140, training loss: 0.061487339437007904
Epoch 150, training loss: 0.06574063003063202
Epoch 160, training loss: 0.06646846979856491
Epoch 170, training loss: 0.06772365421056747
Epoch 180, training loss: 0.07207415997982025
Epoch 190, training loss: 0.06294155865907669
=== picking the best model a

0.8003018108651911

## Node embedding attack

In [9]:
from deeprobust.graph.data import Dataset
from deeprobust.graph.global_attack import NodeEmbeddingAttack
data = Dataset(root='/tmp/', name='cora_ml', seed=15)
adj, features, labels = data.adj, data.features, data.labels
model = NodeEmbeddingAttack()
model.attack(adj, attack_type="remove")
modified_adj = model.modified_adj
model.attack(adj, attack_type="remove", min_span_tree=True)
modified_adj = model.modified_adj
model.attack(adj, attack_type="add", n_candidates=10000)
modified_adj = model.modified_adj
model.attack(adj, attack_type="add_by_remove", n_candidates=10000)
modified_adj = model.modified_adj

Loading cora_ml dataset...
Done!
Selecting 1 largest connected components


## Node embedding victim models

In [10]:
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import DeepWalk
from deeprobust.graph.global_attack import NodeEmbeddingAttack
import numpy as np

dataset_str = 'cora_ml'
data = Dataset(root='/tmp/', name=dataset_str, seed=15)
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test

print("Test DeepWalk on clean graph")
model = DeepWalk(type="skipgram")
model.fit(adj)
print(model.embedding)

Loading cora_ml dataset...
Selecting 1 largest connected components
Test DeepWalk on clean graph


/usr/local/lib/python3.7/dist-packages/numba/core/typed_passes.py:314: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../usr/local/lib/python3.7/dist-packages/deeprobust/graph/defense/node_embedding.py", line 332:
@numba.jit(nopython=True, parallel=True)
def _random_walk(indptr, indices, walk_length, walks_per_node, seed):
^

  state.func_ir.loc))


[[-0.08024385 -0.8360118  -0.47371867 ... -0.00143743  0.46734792
  -0.30357292]
 [ 0.66303325 -0.21016534 -0.6175517  ... -0.06177273  0.26229024
   0.20200323]
 [ 0.5145694  -0.027938   -0.4842296  ...  0.063301    0.16260335
   0.5477671 ]
 ...
 [-0.5318782  -0.07631438 -0.51916534 ...  0.11400343  0.44082317
   0.22816503]
 [ 0.05522015 -0.3257659  -0.20640592 ...  0.63721156 -0.23127624
  -0.5375081 ]
 [-0.35134086  0.264934   -1.0428435  ...  0.5431733  -0.2560392
  -0.10976694]]


In [11]:
print("Test DeepWalk on node classification...")
# model.evaluate_node_classification(labels, idx_train, idx_test, lr_params={"max_iter": 1000})
model.evaluate_node_classification(labels, idx_train, idx_test)
print("Test DeepWalk on link prediciton...")
model.evaluate_link_prediction(adj, np.array(adj.nonzero()).T)

Test DeepWalk on node classification...
Micro F1: 0.8207295373665481
Macro F1: 0.7904730783023871
Test DeepWalk on link prediciton...
ROC error
AUC: 0.0
AP: 1.0


(array([0.9883237 , 0.96201897, 0.98966515, ..., 0.8913061 , 0.8152671 ,
        0.92863464], dtype=float32), 0.0, 1.0)

In [12]:
# set up the attack model
attacker = NodeEmbeddingAttack()
attacker.attack(adj, attack_type="remove", n_perturbations=1000)
modified_adj = attacker.modified_adj
print("Test DeepWalk on attacked graph")
model.fit(modified_adj)
model.evaluate_node_classification(labels, idx_train, idx_test)

Test DeepWalk on attacked graph
Micro F1: 0.7384341637010676
Macro F1: 0.6767085291758695


(array([4, 0, 2, ..., 2, 2, 4]), 0.7384341637010676, 0.6767085291758695)